## Mandate-4 Contributions
### Notebook-1

In Mandate-4, our aim is to imrprove the accuracy of our model which was trained in Mandate-3. In Mandate-3, I trained BART on google PAWS and Quora paraphrasing dataset.
To improve accuracy of my BART paraphrasing model, I have re-trained that model on 'Microsoft Research Paraphrase Corpus' and shuffled PAWS dataset.
I have noted the observations in the end of the notebook.

Install required libraries.
Here, we need transformers version 4.23.1. I have used simpletransformers library to train the model

In [ ]:
!pip install utils
!pip install simpletransformers
!pip install transformers==4.23.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# drive.flush_and_unmount()

Mounted at /content/gdrive


Functions for loading the data and pre-processing the datasets.

In [ ]:
import warnings

import pandas as pd


def load_data(file_path, input_text_column, target_text_column, label_column, keep_label=1):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
        .replace("@", "")
        .replace("#", "")
        .replace("*", "")
    )
    return out_string


Loading MSR paraphrasing dataset and splitting into train and test data.

In [ ]:
from sklearn.model_selection import train_test_split

df = load_data("/content/gdrive/My Drive/msr_paraphrase_train.txt", "#1 String", "#2 String","Quality")

msr_train, msr_test = train_test_split(df)
msr_train = msr_train.head(2100)
msr_test = msr_test.head(1000)



<ipython-input-3-eee7e8d35b0d>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
Skipping line 102: expected 5 fields, saw 6
Skipping line 656: expected 5 fields, saw 6
Skipping line 867: expected 5 fields, saw 6
Skipping line 880: expected 5 fields, saw 6
Skipping line 980: expected 5 fields, saw 6
Skipping line 1439: expected 5 fields, saw 6
Skipping line 1473: expected 5 fields, saw 6
Skipping line 1822: expected 5 fields, saw 6
Skipping line 1952: expected 5 fields, saw 6
Skipping line 2009: expected 5 fields, saw 6
Skipping line 2230: expected 5 fields, saw 6
Skipping line 2506: expected 5 fields, saw 6
Skipping line 2523: expected 5 fields, saw 6
Skipping line 2809: expected 5 fields, saw 6
Skipping line 2887: expected 5 fields, saw 6
Skipping line 2920: expected 5 fields, saw 6
Skipping line 2944: expected 5 fields, saw 6
Skip

In [ ]:
msr_train

,input_text,target_text,prefix
2633,The two have no diplomatic ties and their alre...,The two have no diplomatic ties and their alre...,paraphrase
3165,PeopleSoft also said it expects the acquisitio...,PeopleSoft said the JD Edwards acquisition wou...,paraphrase
1240,They used discarded skin from consenting patie...,Sanders and colleagues at RAFT exposed skin sa...,paraphrase
3332,"In 2002, spending on inpatient hospital care g...","In 2002, spending on inpatient hospital care g...",paraphrase
2877,Mall operators and retailers reported fewer pe...,Mall operators and retailers reported fewer sh...,paraphrase
...,...,...,...
2345,"As of Wednesday, there were 65 probable SARS c...","As of Monday, there were 66 probable cases in ...",paraphrase
3695,The company also became synonymous with ethica...,Levi became known for ethical business practices.,paraphrase
1889,One of the Commission's publicly stated goals ...,The Commission has publicly said one of its go...,paraphrase
2917,"The deal will combine Adobe's Form Server, For...","Big Blue will integrate Adobe's Form Server, F...",paraphrase


In [ ]:
msr_test

,input_text,target_text,prefix
3430,All five suspects were charged with robbery an...,The teens are being held on charges of robbery...,paraphrase
2509,But that amount would probably be impossible t...,Such an amount would probably be unable to sum...,paraphrase
531,He was eventually taken to London's Hammersmit...,"He was taken to Hammersmith hospital, where do...",paraphrase
515,Their difference was over whether the court sh...,Their difference was over whether the court sh...,paraphrase
208,The statistical analysis was published Tuesday...,Their findings were published Monday in Circul...,paraphrase
...,...,...,...
2165,"The Standard & Poor's 500 index declined 6.11,...",The Standard & Poor's 500 index declined by 4....,paraphrase
2198,"Dennehy's family reported him missing June 19,...","Dennehy’s family reported him missing June 19,...",paraphrase
1171,"When the manager let him in the apartment, Bri...","When a manager let him into the apartment, the...",paraphrase
3091,It is probably not the easiest time to come in...,It's probably not the easiest time to come in ...,paraphrase


In [ ]:
msr_train.to_csv("msr_train.tsv", sep="\t")
msr_test.to_csv("msr_test.tsv", sep="\t")

In [ ]:
!ls -l

total 684
drwx------ 5 root root   4096 May  3 10:54 gdrive
-rw-r--r-- 1 root root 169529 May  3 11:59 msr_test.tsv
-rw-r--r-- 1 root root 520101 May  3 11:59 msr_train.tsv
drwxr-xr-x 1 root root   4096 May  1 13:36 sample_data


In [ ]:

train_df = msr_train[["prefix", "input_text", "target_text"]]
eval_df = msr_test[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

print(train_df)

          prefix                                         input_text  \
3259  paraphrase  Drug developer Inspire Pharmaceuticals Inc. IS...   
257   paraphrase  On Sunday, the experts will perform the first ...   
1918  paraphrase  But the institute says the department "woefull...   
2145  paraphrase  But MTA officials appropriated the money to th...   
3848  paraphrase  The blue-chip Dow Jones industrial average.DJI...   
...          ...                                                ...   
3423  paraphrase  SEC Chairman William Donaldson said there is a...   
1533  paraphrase  In December 1998, a 33-year-old woman died whe...   
1472  paraphrase  But Gelinas says only six have been fully re-e...   
1098  paraphrase  The ISM noted that "with the exception of Marc...   
2210  paraphrase  Bombardier and Embraer will each deliver 85 je...   

                                            target_text  
3259  Drug developer Inspire Pharmaceuticals Inc. (n...  
257   Today, the experts will p

<ipython-input-6-afc17de22461>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
<ipython-input-6-afc17de22461>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)


Now we will set up the model and hyperparameter values.
We will train for 3 epochs. 

In [ ]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs


model_args = Seq2SeqArgs()
model_args.eval_batch_size = 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_seq_length = 128
model_args.num_train_epochs = 3
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.do_sample = True
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.max_length = 128
model_args.top_k = 50
model_args.top_p = 0.95


Here, best_model is the model which was fine-tuned on PAWS and Quora dataset in previous mandate. Now, we will start training.

In [ ]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="/content/gdrive/My Drive/best_model",
    args=model_args
) 

model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/1985 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/249 [00:00<?, ?it/s]

  0%|          | 0/661 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/249 [00:00<?, ?it/s]

  0%|          | 0/661 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/249 [00:00<?, ?it/s]

  0%|          | 0/661 [00:00<?, ?it/s]

(747,
 {'global_step': [249, 498, 747],
  'eval_loss': [0.2942070717161352, 0.27673506194894965, 0.283113723451441],
  'train_loss': [0.44113749265670776,
   0.21089355647563934,
   0.10880301147699356]})

Saving the model to google drive

In [ ]:
# !pwd
# !cd /content/outputs && ls -l
# !cd /content/gdrive/MyDrive/ && ls -l
!ls -l
# !date
# !cp -r "/content/outputs/best_model" "/content/gdrive/My Drive/2MayNewModel"
# !cp -r "/content/msr_test.tsv" "/content/gdrive/My Drive/2MayNewModel"
# !cp -r "/content/msr_train.tsv" "/content/gdrive/My Drive/2MayNewModel"

total 696
drwxr-xr-x 2 root root   4096 May  2 10:57 cache_dir
drwx------ 5 root root   4096 May  2 10:54 gdrive
-rw-r--r-- 1 root root 170056 May  2 10:54 msr_test.tsv
-rw-r--r-- 1 root root 519574 May  2 10:54 msr_train.tsv
drwxr-xr-x 6 root root   4096 May  2 11:11 outputs
drwxr-xr-x 3 root root   4096 May  2 10:57 runs
drwxr-xr-x 1 root root   4096 Apr 28 13:35 sample_data


Testing the new model which we trained now :--

In [ ]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

# model = Seq2SeqModel(encoder_decoder_type="bart", encoder_decoder_name="outputs/best_model")
model = Seq2SeqModel(encoder_decoder_type="bart", 
                     encoder_decoder_name="/content/gdrive/My Drive/2MayNewModel/best_model"
                     ,use_cuda=False
                     )

to_predict = ["They were there to enjoy us and they were there to pray for us.",
              "What is the best way to play Cricket?",
              "This is the best way to create food.",
              "What are different ways to play badminton?",
              "We should not talk in rude manner to guests.",
              "I do not know how many times I have prepared this model.",
              "This is how you make cake",
              "We remove special symbols like #, @, *, etc. We cannot remove useful punctuations like question mark because the paraphrase of a question will be a question only. We use  regex or python string functions.",
              "Amrozi accused his brother, whom he called \"the witness\", of deliberately distorting  his evidence.",
              "What are the ways to commit suicide?",
              "How can I get started using Quora?",
              "How to play chess?"
              ]

# while True:
for i in to_predict:
    to_predict1 = [i]

    preds = model.predict(to_predict1)

    print("---------------------------------------------------------")
    print(i)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)

    print("---------------------------------------------------------")
    print()

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
They were there to enjoy us and they were there to pray for us.What is the best way to play Cricket?

Predictions >>>
They were there to enjoy us and they were there for us, he said, adding that they are there to pray for us.
They were there to enjoy us and they were there also to pray for us, he said.
They were there to enjoy us and they were there also to pray for us, he said.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
This is the best way to create food.

Predictions >>>
This is the best way to create food.
This is the best way to create food.
This is the best way to create food.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

---------------------------------------------------------
What are the ways to commit suicide?

Predictions >>>
What are the most insane ways to commit suicide?
What are the most insane ways to commit suicide?
What are the most insane ways to commit suicide?
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
How can I get started using Quora?

Predictions >>>
How can I get started with Quora?
How can I get started with Quora?
How can I get started with Quora?
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
How to play chess?

Predictions >>>
How do I play chess?
How do I play chess?
How do I play chess?
---------------------------------------------------------



Outputs of the model which was trained in previous mandate:--

In [ ]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

# model = Seq2SeqModel(encoder_decoder_type="bart", encoder_decoder_name="outputs/best_model")
model = Seq2SeqModel(encoder_decoder_type="bart", 
                     encoder_decoder_name="/content/gdrive/My Drive/best_model"
                     ,use_cuda=False
                     # use_cuda=True if you have GPU access else assign it False
                     )

to_predict = ["They were there to enjoy us and they were there to pray for us.",
              "What is the best way to play Cricket?",
              "This is the best way to create food.",
              "What are different ways to play badminton?",
              "We should not talk in rude manner to guests.",
              "I do not know how many times I have prepared this model.",
              "This is how you make cake",
              "We remove special symbols like #, @, *, etc. We cannot remove useful punctuations like question mark because the paraphrase of a question will be a question only. We use  regex or python string functions.",
              "Amrozi accused his brother, whom he called \"the witness\", of deliberately distorting  his evidence.",
              "What are the ways to commit suicide?",
              "How can I get started using Quora?",
              "How to play chess?"
              ]

# while True:
for i in to_predict:
    to_predict1 = [i]

    preds = model.predict(to_predict1)

    print("---------------------------------------------------------")
    print(i)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)

    print("---------------------------------------------------------")
    print()

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
They were there to enjoy us and they were there to pray for us.

Predictions >>>
They were there to enjoy us and they were there praying for us.
They were there to enjoy us and they were there also to pray for us.
They were there to enjoy us and they were there praying for us.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
What is the best way to play Cricket?

Predictions >>>
How do I play cricket?
What is the best way to play cricket?
How do I play cricket?
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

---------------------------------------------------------
What are the ways to commit suicide?

Predictions >>>
How can I commit suicide?
How can I commit suicide?
How can I commit suicide?
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
How can I get started using Quora?

Predictions >>>
How do I get started with Quora?
How do I get started with Quora?
How do I get started with Quora?
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
How to play chess?

Predictions >>>
How do I play chess?
How do I play chess?
How do I play chess?
---------------------------------------------------------



Here, from the outputs, we can observe that after getting trained on MSR, some extra phrases like 'he said.' etc are added. 
Model still generates a question for an assertive sentence. The model removes extra spaces or adds articles. In the next notebook, finetune this model again.
